In [1]:
import keepa
import pandas as pd
import time
import inspect
import csv
from datetime import date
from datetime import datetime

In [2]:
accesskey = 'gfe071s7bsqhvf126ac7r1v7ug770q8uu1rtgleh4tprpq3oesuuqvatfo7lii50'
api = keepa.Keepa(accesskey)

In [3]:
#Historical data retrieved from keepa are in dataframe format, sometimes with multiple observations for each date
#Sicne amazone does not provide information of time in review, a simple average was used to calculated
def transform_df(df):
    t_df = df
    try:
        t_df.index = df.index.strftime('%Y-%m-%d')
    except:
        print('date already formatted')
#-1 means missing. This is removed first
    t_df = t_df[t_df.value != -1]
    t_df = t_df.groupby(t_df.index).mean()
    return t_df

In [5]:
def retrieve_data(asin, products):
    if len(products[0]['data']) != 0:
        #price cheapest new offering
        price_new_history = transform_df(products[0]['data']['df_NEW']).rename({'value' : 'price_new'}, axis=1)
        #price cheapest used product offering
        price_used_history = transform_df(products[0]['data']['df_USED']).rename({'value' : 'price_used'}, axis=1)
        #Star ratings
        try:
            rating_history = transform_df(products[0]['data']['df_RATING']).rename({'value' : 'rating'}, axis=1)
        except:
            rating_history = pd.DataFrame({'rating' : None}, index=[date.today()])
        #Number of reviews
        try:
            review_history = transform_df(products[0]['data']['df_COUNT_REVIEWS']).rename({'value' : 'reviews'}, axis=1)
        except:
            review_history = pd.DataFrame({'reviews' : None}, index=[date.today()])
        #Merge everything with outer joins
        df = price_new_history.merge(price_used_history, right_index = True, left_index = True, how = "outer").merge(
        rating_history, right_index = True, left_index = True, how = "outer").merge(
        review_history, right_index = True, left_index = True, how = "outer")
    #If there's no historical data
    else:
        df = pd.DataFrame({'price_new':products[0]['stats']['avg'][1], 
                           'price_used' : products[0]['stats']['avg'][2], 
                           'rating' : None, 
                           'reviews' : None}, index=[date.today()])
    df["asin"] = asin
    #Out of stock new products
    df["oos_new"] = products[0]['stats']["outOfStockPercentageInInterval"][1]
    #Out of stock used products
    df["oos_used"] = products[0]['stats']["outOfStockPercentageInInterval"][2]
    #Number of keepa minutes since product was listed
    df["listed_since"] = products[0]['listedSince']
    df.index = pd.to_datetime(df.index)
    #df = df.interpolate(method='time')
    return df

In [6]:
#import data from csv file
data = pd.read_csv('../../gen/input/amazon_search_hedonic_clean.csv', sep = ";")
asins = data['asin'].unique()

In [34]:
#with open(f"../../gen/input/api_info.csv", "w", encoding = "UTF-8", newline = '') as csv_file:
    writer = csv.writer(csv_file, delimiter = ";")
    writer.writerow(["date", "price_new", "price_used", "rating", "reviews", "asin", "oos_new", "oos_used", "listed_since"])

In [61]:
data_so_far = pd.read_csv(f"../../gen/input/api_info.csv", sep = ";")
asins_so_far = data_so_far['asin'].unique()

print(len(asins))
print(len(asins_so_far))
asins_left = list(set(asins) - set(asins_so_far))
len(asins_left)


2291
223


2068

In [65]:
#get number of tokens left
round((api.tokens_left/2))

26

In [14]:
data_so_far = pd.read_csv(f"../../gen/input/api_info.csv", sep = ";")
asins_so_far = data_so_far['asin'].unique()

print(len(asins))
print(len(asins_so_far))
asins_left = list(set(asins) - set(asins_so_far))
print(len(asins_left))

i = 0
tokens_left = api.tokens_left
print("tokens left: ", tokens_left)
for asin in asins_left[1:round(((tokens_left*1.1)/2))]:
    print(asin)
    products = api.query(asin, stats = 3650, history = 1, rating = 1)
    df = retrieve_data(asin, products)
    #df1 = df.interpolate(method='time')
    #df2 = df1.fillna(method='bfill')
    df.to_csv(f"../../gen/input/api_info.csv", mode = "a", sep = ";", header = False)
    i += 1
    if i % 10 == 0:
        print(i)
    time.sleep(0.3)

2291
626
1665
tokens left:  90
B0C5CTHCD3


100%|██████████| 1/1 [00:00<00:00,  3.28it/s]


B099RP92P5


100%|██████████| 1/1 [00:01<00:00,  1.42s/it]


B071HP7795


...

100%|██████████| 1/1 [00:02<00:00,  2.20s/it]


B0CNS5KR56


100%|██████████| 1/1 [00:03<00:00,  3.06s/it]


B0CQ9SZJN6


100%|██████████| 1/1 [00:04<00:00,  4.44s/it]


# testser

In [ ]:
products = api.query("B09BFSCCNB", stats = 3650, history = 1, rating = 1, stock = 1)

In [10]:
df = retrieve_data("B09BFSCCNB", products)

In [11]:
df

,price_new,price_used,rating,reviews,asin,oos_new,oos_used,listed_since
2021-07-29,NaN,NaN,NaN,NaN,B09BFSCCNB,17,25,5800320
2022-01-11,769.99,NaN,NaN,NaN,B09BFSCCNB,17,25,5800320
2022-01-13,NaN,NaN,4.6,7896.666667,B09BFSCCNB,17,25,5800320
2022-01-14,NaN,NaN,NaN,7909.666667,B09BFSCCNB,17,25,5800320
2022-01-15,NaN,NaN,NaN,7920.750000,B09BFSCCNB,17,25,5800320
...,...,...,...,...,...,...,...,...
2024-04-25,583.94,NaN,NaN,14758.000000,B09BFSCCNB,17,25,5800320
2024-04-26,589.02,NaN,NaN,14759.500000,B09BFSCCNB,17,25,5800320
2024-04-27,NaN,346.84,NaN,NaN,B09BFSCCNB,17,25,5800320
2024-04-28,677.03,368.13,NaN,14760.000000,B09BFSCCNB,17,25,5800320


In [12]:
#linearly interpolate missing values in df
df1 = df.interpolate(method='time')

In [16]:
df1

,price_new,price_used,rating,reviews,asin,oos_new,oos_used,listed_since
2021-07-29,NaN,NaN,NaN,NaN,B09BFSCCNB,17,25,5800320
2022-01-11,769.990,NaN,NaN,NaN,B09BFSCCNB,17,25,5800320
2022-01-13,759.990,NaN,4.6,7896.666667,B09BFSCCNB,17,25,5800320
2022-01-14,754.990,NaN,4.6,7909.666667,B09BFSCCNB,17,25,5800320
2022-01-15,749.990,NaN,4.6,7920.750000,B09BFSCCNB,17,25,5800320
...,...,...,...,...,...,...,...,...
2024-04-25,583.940,336.1950,4.5,14758.000000,B09BFSCCNB,17,25,5800320
2024-04-26,589.020,341.5175,4.5,14759.500000,B09BFSCCNB,17,25,5800320
2024-04-27,633.025,346.8400,4.5,14759.750000,B09BFSCCNB,17,25,5800320
2024-04-28,677.030,368.1300,4.5,14760.000000,B09BFSCCNB,17,25,5800320


In [17]:
#backwards fill missing values in df
df2 = df1.fillna(method='bfill')

In [18]:
df2

,price_new,price_used,rating,reviews,asin,oos_new,oos_used,listed_since
2021-07-29,769.990,615.9900,4.6,7896.666667,B09BFSCCNB,17,25,5800320
2022-01-11,769.990,615.9900,4.6,7896.666667,B09BFSCCNB,17,25,5800320
2022-01-13,759.990,615.9900,4.6,7896.666667,B09BFSCCNB,17,25,5800320
2022-01-14,754.990,615.9900,4.6,7909.666667,B09BFSCCNB,17,25,5800320
2022-01-15,749.990,615.9900,4.6,7920.750000,B09BFSCCNB,17,25,5800320
...,...,...,...,...,...,...,...,...
2024-04-25,583.940,336.1950,4.5,14758.000000,B09BFSCCNB,17,25,5800320
2024-04-26,589.020,341.5175,4.5,14759.500000,B09BFSCCNB,17,25,5800320
2024-04-27,633.025,346.8400,4.5,14759.750000,B09BFSCCNB,17,25,5800320
2024-04-28,677.030,368.1300,4.5,14760.000000,B09BFSCCNB,17,25,5800320
